In [1]:
import argparse
import csv
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from keras.layers import Input, Embedding, LSTM, Dense
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import csv
import nltk
import re
from gensim.models import Word2Vec
import pandas as pd 
import numpy as np
from sklearn.datasets import load_files
from collections import Counter 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.porter import PorterStemmer
import gc

C:\Users\Youri\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
movie_reviews = pd.read_csv("IMDB Dataset.csv")
movie_reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
text= movie_reviews['review']
X=text.values.tolist()

In [4]:
def majid(X):
    corpus = []
    for i in range(0, len(X)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(X[i])) #remove punctuation
        review = re.sub(r'\d+',' ', str(X[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)        
#    return corpus        
    #Tokenizing and Word Count  
    words=[]
    for i in range(len(corpus)):
        words= nltk.word_tokenize(corpus[i])
        #sentences.append(words)
   
    return words

X = [[el] for el in X] 

In [5]:
X[0]

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [6]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()
sentences = Parallel(n_jobs=num_cores)(delayed(majid)(i) for i in X)

In [7]:
#############
#Porter stemmer
#############
def porter_stemming(sentences):
        porter = PorterStemmer()
        #words = sent.split() 
        stemmed_words = [porter.stem(word) for word in sentences]
        return stemmed_words
def majid_porter(X):
    sentences= porter_stemming(X)
    gc.collect()
    return sentences
 
from joblib import Parallel, delayed
import multiprocessing
#X = [[el] for el in X]
    
num_cores = multiprocessing.cpu_count()
porter = Parallel(n_jobs=num_cores)(delayed(majid_porter)(i) for i in sentences)

In [8]:
#######################
#Snowball Stemmer
########################
from nltk.stem.snowball import SnowballStemmer

import nltk
def snowball_stemming(sentences):
        sno = nltk.stem.SnowballStemmer('english')
        #words = sent.split() 
        stemmed_words = [sno.stem(word) for word in sentences]
        return stemmed_words

def majid_snowball(X):
    sentences= snowball_stemming(X)
    gc.collect()
    return sentences
 
from joblib import Parallel, delayed
import multiprocessing

#X = [[el] for el in X]
    
num_cores = multiprocessing.cpu_count()
snowball = Parallel(n_jobs=num_cores)(delayed(majid_snowball)(i) for i in sentences)

In [9]:
#######################
#Lancaster Stemmer
########################
from nltk.stem import LancasterStemmer

import nltk
def lancaster_stemming(sentences):
    lancaster = LancasterStemmer()
    #words = sent.split() 
    stemmed_words = [lancaster.stem(word) for word in sentences]
    return stemmed_words

def majid_lancaster(X):
    sentences= lancaster_stemming(X)
    gc.collect()
    return sentences
 
from joblib import Parallel, delayed
import multiprocessing

#X = [[el] for el in X]
    
num_cores = multiprocessing.cpu_count()
lancaster = Parallel(n_jobs=num_cores)(delayed(majid_lancaster)(i) for i in sentences)

In [10]:
#######################
#Lemmatization
########################
from nltk.stem import WordNetLemmatizer

import nltk
def lemmaztization(sentences):
    WNlemmatizer = WordNetLemmatizer()
    #words = sent.split() 
    lemmas = [WNlemmatizer.lemmatize(word) for word in sentences]
    return lemmas

def majid_lemmatization(X):
    sentences= lemmaztization(X)
    gc.collect()
    return sentences
 
from joblib import Parallel, delayed
import multiprocessing

#X = [[el] for el in X]
    
num_cores = multiprocessing.cpu_count()
lemma = Parallel(n_jobs=num_cores)(delayed(majid_lemmatization)(i) for i in sentences)

In [11]:
X_porter = [' '.join(x) for x in porter]
#sent4= TreebankWordDetokenizer().detokenize([ i for i in [sent3]])

#after preprocess our reviews we will store them in a new list 

In [12]:
X_snowball = [' '.join(x) for x in snowball]
#sent4= TreebankWordDetokenizer().detokenize([ i for i in [sent3]])

#after preprocess our reviews we will store them in a new list 

In [13]:
X_lancaster = [' '.join(x) for x in lancaster]
#sent4= TreebankWordDetokenizer().detokenize([ i for i in [sent3]])

#after preprocess our reviews we will store them in a new list 

In [14]:
X_lemma = [' '.join(x) for x in lemma]
#sent4= TreebankWordDetokenizer().detokenize([ i for i in [sent3]])

#after preprocess our reviews we will store them in a new list 

In [15]:
print(X_porter[0])

one of the other review ha mention that after watch just oz episod you 'll be hook . they are right , as thi is exactli what happen with me.th first thing that struck me about oz wa it brutal and unflinch scene of violenc , which set in right from the word go . trust me , thi is not a show for the faint heart or timid . thi show pull no punch with regard to drug , sex or violenc . it is hardcor , in the classic use of the word.it is call oz as that is the nicknam given to the oswald maximum secur state penitentari . it focus mainli on emerald citi , an experiment section of the prison where all the cell have glass front and face inward , so privaci is not high on the agenda . em citi is home to mani .. aryan , muslim , gangsta , latino , christian , italian , irish and more .... so scuffl , death stare , dodgi deal and shadi agreement are never far away.i would say the main appeal of the show is due to the fact that it goe where other show would n't dare . forget pretti pictur paint fo

In [16]:
#we need to convert our labels into digits    
y = movie_reviews['sentiment']

y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))
#Y = pd.get_dummies(df['Product']).values
#print('Shape of label tensor:', Y.shape)

In [17]:
#We  use train_test_split method from the sklearn.model.selection
Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_porter, y, test_size=0.20, random_state=42) #porter
Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_snowball, y, test_size=0.20, random_state=42) #snowball
Xla_train, Xla_test, yla_train, yla_test = train_test_split(X_lancaster, y, test_size=0.20, random_state=42) #lancaster
Xle_train, Xle_test, yle_train, yle_test = train_test_split(X_lemma, y, test_size=0.20, random_state=42) #snowball

In [18]:
#Preparing the Embedding Layer
#use the Tokenizer class from the keras.preprocessing.
#text module to create a word-to-index dictionary.
#In the word-to-index dictionary, each word in the corpus is used as a key, 
#while a corresponding unique index is used as the value for the key
def prepare_embedding(X_train, X_test):
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    # Adding 1 because of reserved 0 index
    vocab_size = len(tokenizer.word_index) + 1
    print('Vocab size:',vocab_size)
    maxlen = 100

    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    return X_train, X_test, vocab_size, tokenizer

In [19]:
Xp_train, Xp_test, vocab_size_p, tokenizer_p = prepare_embedding(Xp_train, Xp_test) #porter
Xs_train, Xs_test, vocab_size_s, tokenizer_s = prepare_embedding(Xs_train, Xs_test) #snowball
Xla_train, Xla_test, vocab_size_la, tokenizer_la = prepare_embedding(Xla_train, Xla_test) #lancaster
Xle_train, Xle_test, vocab_size_le, tokenizer_le = prepare_embedding(Xle_train, Xle_test) #lemma

Vocab size: 78012
Vocab size: 73690
Vocab size: 72586
Vocab size: 91915


In [20]:
#We use pretrained embeddings to create our feature matrix. 
#In the following script we load the pretrained word embeddings 
#and create a dictionary that will contain words as keys and 
#their corresponding embedding list as values.
from numpy import array
from numpy import asarray
from numpy import zeros
from gensim.models import Word2Vec

def prepare_embedding_layer(embedding_file, vocab_size, tokenizer):
    embeddings_dictionary = dict()
    glove_file = open(embedding_file, encoding="utf8")

    for line in glove_file:
        records = line.split()
        word = records[0]
        vector_dimensions = asarray(records[1:])
        embeddings_dictionary [word] = vector_dimensions
    glove_file.close()
    
    embedding_matrix = zeros((vocab_size, 300))
    for word, index in tokenizer.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    
    return embedding_matrix

In [21]:
embedding_matrix_CBOWporter = prepare_embedding_layer("W-CBOW-Stem-porter.txt", vocab_size_p, tokenizer_p)

In [22]:
embedding_matrix_CBOWsnowball = prepare_embedding_layer("W-CBOW-Stem-snowball.txt", vocab_size_s, tokenizer_s)

In [23]:
embedding_matrix_CBOWlancaster = prepare_embedding_layer("W-CBOW-Stem-lancaster.txt", vocab_size_la, tokenizer_la)

In [24]:
embedding_matrix_CBOWlemma = prepare_embedding_layer("W-CBOW-Stem-lemma.txt", vocab_size_le, tokenizer_le)

In [25]:
embedding_matrix_SKIPporter = prepare_embedding_layer("W-Skip-Stem-porter.txt", vocab_size_p, tokenizer_p)

In [26]:
embedding_matrix_SKIPsnowball = prepare_embedding_layer("W-Skip-Stem-snowball.txt", vocab_size_s, tokenizer_s)

In [27]:
embedding_matrix_SKIPlancaster = prepare_embedding_layer("W-Skip-Stem-lancaster.txt", vocab_size_la, tokenizer_la)

In [28]:
embedding_matrix_SKIPlemma = prepare_embedding_layer("W-Skip-Stem-lemma.txt", vocab_size_le, tokenizer_le)

In [29]:
# define and fit the model

def get_model(X_train, y_train, vocab_size, embedding_matrix, maxlen=100):
	model = Sequential()
	embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
	model.add(embedding_layer)
	model.add(LSTM(128))
	model.add(Dense(1, activation='sigmoid'))
	# define model
	#model = Sequential()
	#model.add(Dense(100, input_dim=2, activation='relu'))
	#model.add(Dense(1, activation='sigmoid'))
	# compile model
	model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
	# fit model
	model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.10)
	return model

####################

# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# fit the model
#history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.10)

############

In [30]:
model_CBOWporter = get_model(Xp_train, yp_train, vocab_size_p, embedding_matrix_CBOWporter)

Epoch 1/6
282/282 [==============================] - 58s 146ms/step - loss: 0.5576 - acc: 0.7017 - val_loss: 0.3990 - val_acc: 0.8142
Epoch 2/6
282/282 [==============================] - 38s 134ms/step - loss: 0.3791 - acc: 0.8319 - val_loss: 0.3751 - val_acc: 0.8265
Epoch 3/6
282/282 [==============================] - 37s 131ms/step - loss: 0.3079 - acc: 0.8694 - val_loss: 0.4038 - val_acc: 0.8092
Epoch 4/6
282/282 [==============================] - 39s 137ms/step - loss: 0.2497 - acc: 0.8981 - val_loss: 0.3686 - val_acc: 0.8397
Epoch 5/6
282/282 [==============================] - 38s 133ms/step - loss: 0.2000 - acc: 0.9207 - val_loss: 0.3678 - val_acc: 0.8395
Epoch 6/6
282/282 [==============================] - 38s 134ms/step - loss: 0.1565 - acc: 0.9452 - val_loss: 0.3925 - val_acc: 0.8453


In [31]:
model_CBOWsnowball = get_model(Xs_train, ys_train, vocab_size_s, embedding_matrix_CBOWsnowball)

Epoch 1/6
282/282 [==============================] - 38s 127ms/step - loss: 0.5752 - acc: 0.6841 - val_loss: 0.4152 - val_acc: 0.8090
Epoch 2/6
282/282 [==============================] - 36s 127ms/step - loss: 0.3690 - acc: 0.8405 - val_loss: 0.3634 - val_acc: 0.8375
Epoch 3/6
282/282 [==============================] - 35s 126ms/step - loss: 0.2964 - acc: 0.8756 - val_loss: 0.3508 - val_acc: 0.8435
Epoch 4/6
282/282 [==============================] - 36s 126ms/step - loss: 0.2407 - acc: 0.9051 - val_loss: 0.3478 - val_acc: 0.8455
Epoch 5/6
282/282 [==============================] - 36s 127ms/step - loss: 0.1937 - acc: 0.9271 - val_loss: 0.3742 - val_acc: 0.8503
Epoch 6/6
282/282 [==============================] - 36s 126ms/step - loss: 0.1409 - acc: 0.9513 - val_loss: 0.3912 - val_acc: 0.8455


In [32]:
model_CBOWlancaster = get_model(Xla_train, yla_train, vocab_size_la, embedding_matrix_CBOWlancaster)

Epoch 1/6
282/282 [==============================] - 39s 129ms/step - loss: 0.5723 - acc: 0.6911 - val_loss: 0.4189 - val_acc: 0.8048
Epoch 2/6
282/282 [==============================] - 35s 125ms/step - loss: 0.3826 - acc: 0.8322 - val_loss: 0.4051 - val_acc: 0.8165
Epoch 3/6
282/282 [==============================] - 36s 126ms/step - loss: 0.3210 - acc: 0.8633 - val_loss: 0.3674 - val_acc: 0.8347
Epoch 4/6
282/282 [==============================] - 36s 129ms/step - loss: 0.2602 - acc: 0.8909 - val_loss: 0.4367 - val_acc: 0.8000
Epoch 5/6
282/282 [==============================] - 35s 124ms/step - loss: 0.2136 - acc: 0.9153 - val_loss: 0.3884 - val_acc: 0.8430
Epoch 6/6
282/282 [==============================] - 35s 124ms/step - loss: 0.1645 - acc: 0.9397 - val_loss: 0.4260 - val_acc: 0.8418


In [33]:
model_CBOWlemma = get_model(Xle_train, yle_train, vocab_size_le, embedding_matrix_CBOWlemma)

Epoch 1/6
282/282 [==============================] - 39s 129ms/step - loss: 0.5762 - acc: 0.6870 - val_loss: 0.4074 - val_acc: 0.8080
Epoch 2/6
282/282 [==============================] - 36s 128ms/step - loss: 0.3769 - acc: 0.8373 - val_loss: 0.3882 - val_acc: 0.8142
Epoch 3/6
282/282 [==============================] - 36s 129ms/step - loss: 0.3075 - acc: 0.8692 - val_loss: 0.3906 - val_acc: 0.8307
Epoch 4/6
282/282 [==============================] - 36s 127ms/step - loss: 0.2461 - acc: 0.8995 - val_loss: 0.3551 - val_acc: 0.8457
Epoch 5/6
282/282 [==============================] - 36s 127ms/step - loss: 0.1936 - acc: 0.9262 - val_loss: 0.3891 - val_acc: 0.8465
Epoch 6/6
282/282 [==============================] - 35s 124ms/step - loss: 0.1472 - acc: 0.9485 - val_loss: 0.3960 - val_acc: 0.8537


In [34]:
model_SKIPporter = get_model(Xp_train, yp_train, vocab_size_p, embedding_matrix_SKIPporter)

Epoch 1/6
282/282 [==============================] - 38s 127ms/step - loss: 0.5833 - acc: 0.6722 - val_loss: 0.4283 - val_acc: 0.8015
Epoch 2/6
282/282 [==============================] - 35s 125ms/step - loss: 0.4278 - acc: 0.8038 - val_loss: 0.4673 - val_acc: 0.7688
Epoch 3/6
282/282 [==============================] - 38s 134ms/step - loss: 0.3977 - acc: 0.8178 - val_loss: 0.4661 - val_acc: 0.8018
Epoch 4/6
282/282 [==============================] - 38s 135ms/step - loss: 0.3625 - acc: 0.8428 - val_loss: 0.3494 - val_acc: 0.8512
Epoch 5/6
282/282 [==============================] - 37s 130ms/step - loss: 0.3271 - acc: 0.8579 - val_loss: 0.3545 - val_acc: 0.8508
Epoch 6/6
282/282 [==============================] - 39s 137ms/step - loss: 0.3071 - acc: 0.8655 - val_loss: 0.3267 - val_acc: 0.8645


In [35]:
model_SKIPsnowball = get_model(Xs_train, ys_train, vocab_size_s, embedding_matrix_SKIPsnowball)

Epoch 1/6
282/282 [==============================] - 42s 138ms/step - loss: 0.5872 - acc: 0.6724 - val_loss: 0.5449 - val_acc: 0.7498
Epoch 2/6
282/282 [==============================] - 38s 137ms/step - loss: 0.4526 - acc: 0.7934 - val_loss: 0.4137 - val_acc: 0.8085
Epoch 3/6
282/282 [==============================] - 39s 137ms/step - loss: 0.4057 - acc: 0.8154 - val_loss: 0.4047 - val_acc: 0.8290
Epoch 4/6
282/282 [==============================] - 39s 138ms/step - loss: 0.3655 - acc: 0.8359 - val_loss: 0.3753 - val_acc: 0.8260
Epoch 5/6
282/282 [==============================] - 41s 146ms/step - loss: 0.3419 - acc: 0.8472 - val_loss: 0.3648 - val_acc: 0.8393
Epoch 6/6
282/282 [==============================] - 37s 133ms/step - loss: 0.3221 - acc: 0.8594 - val_loss: 0.3411 - val_acc: 0.8530


In [36]:
model_SKIPlancaster = get_model(Xla_train, yla_train, vocab_size_la, embedding_matrix_SKIPlancaster)

Epoch 1/6
282/282 [==============================] - 40s 135ms/step - loss: 0.6069 - acc: 0.6649 - val_loss: 0.4464 - val_acc: 0.7925
Epoch 2/6
282/282 [==============================] - 37s 131ms/step - loss: 0.4397 - acc: 0.7961 - val_loss: 0.4529 - val_acc: 0.7905
Epoch 3/6
282/282 [==============================] - 37s 133ms/step - loss: 0.4134 - acc: 0.8117 - val_loss: 0.3979 - val_acc: 0.8195
Epoch 4/6
282/282 [==============================] - 38s 134ms/step - loss: 0.3778 - acc: 0.8305 - val_loss: 0.3713 - val_acc: 0.8330
Epoch 5/6
282/282 [==============================] - 37s 131ms/step - loss: 0.3480 - acc: 0.8456 - val_loss: 0.3746 - val_acc: 0.8450
Epoch 6/6
282/282 [==============================] - 37s 132ms/step - loss: 0.3359 - acc: 0.8530 - val_loss: 0.4231 - val_acc: 0.8055


In [37]:
model_SKIPlemma = get_model(Xle_train, yle_train, vocab_size_le, embedding_matrix_SKIPlemma)

Epoch 1/6
282/282 [==============================] - 41s 139ms/step - loss: 0.5721 - acc: 0.6904 - val_loss: 0.4222 - val_acc: 0.8055
Epoch 2/6
282/282 [==============================] - 37s 131ms/step - loss: 0.4269 - acc: 0.8050 - val_loss: 0.4124 - val_acc: 0.8065
Epoch 3/6
282/282 [==============================] - 38s 133ms/step - loss: 0.3893 - acc: 0.8239 - val_loss: 0.3716 - val_acc: 0.8360
Epoch 4/6
282/282 [==============================] - 38s 136ms/step - loss: 0.3574 - acc: 0.8429 - val_loss: 0.3550 - val_acc: 0.8460
Epoch 5/6
282/282 [==============================] - 37s 132ms/step - loss: 0.3224 - acc: 0.8618 - val_loss: 0.3594 - val_acc: 0.8457
Epoch 6/6
282/282 [==============================] - 39s 138ms/step - loss: 0.2973 - acc: 0.8738 - val_loss: 0.3330 - val_acc: 0.8535


In [38]:
''''
from keras import backend as K
def recall_m(y_train, y_test):
        true_positives = K.sum(K.round(K.clip(y_train * y_test, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_train, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def precision_m(y_train, y_test):
        true_positives = K.sum(K.round(K.clip(y_train * y_test, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def f1_m(y_train, y_test):
    precision = precision_m(y_train, y_test)
    recall = recall_m(y_train, y_test)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
'''
'''
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_test, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
####
    '''
# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# fit the model
#history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.10)

'\nfrom keras import backend as K\ndef f1(y_true, y_pred):\n    def recall(y_test, y_pred):\n        """Recall metric.\n        Only computes a batch-wise average of recall.\n        Computes the recall, a metric for multi-label classification of\n        how many relevant items are selected.\n        """\n        true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))\n        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))\n        recall = true_positives / (possible_positives + K.epsilon())\n        return recall\n    def precision(y_true, y_pred):\n        """Precision metric.\n        Only computes a batch-wise average of precision.\n        Computes the precision, a metric for multi-label classification of\n        how many selected items are relevant.\n        """\n        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))\n        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))\n        precision = true_positives / (predicted_positives + 

In [39]:
############
#from sklearn.metrics import classification_report
#from sklearn.metrics import precision_recall_fscore_support
#y_pred = model.predict(X_test)
#print(precision_recall_fscore_support(y_test, y_pred))

from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from keras.models import Sequential
from keras.layers import Dense

def evaluate(model, X_test, y_test):
    yhat_probs = model.predict(X_test, verbose=1)
    # predict crisp classes for test set
    yhat_classes = model.predict_classes(X_test, verbose=1)
    #yhat_classes = np.argmax(model.predict(X_test) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]
 
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    #print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))
    #print(classification_report(y_test, y_pred, average='weighted'))


    # evaluate the model
    #loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)

In [40]:
evaluate(model_CBOWporter, Xp_test, yp_test)

  5/313 [..............................] - ETA: 9s

C:\Users\Youri\anaconda3\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


313/313 [==============================] - 7s 23ms/step
Accuracy: 0.852600
Precision: 0.872986
Recall: 0.827942
F1 score: 0.849868
[[4354  607]
 [ 867 4172]]


In [41]:
evaluate(model_CBOWsnowball, Xs_test, ys_test)

313/313 [==============================] - 7s 24ms/step
Accuracy: 0.850200
Precision: 0.820221
Recall: 0.899980
F1 score: 0.858251
[[3967  994]
 [ 504 4535]]


In [42]:
evaluate(model_CBOWlancaster, Xla_test, yla_test)

313/313 [==============================] - 7s 23ms/step
Accuracy: 0.853100
Precision: 0.830556
Recall: 0.890058
F1 score: 0.859278
[[4046  915]
 [ 554 4485]]


In [43]:
evaluate(model_CBOWlemma, Xle_test, yle_test)

313/313 [==============================] - 7s 23ms/step
Accuracy: 0.854600
Precision: 0.845575
Recall: 0.870411
F1 score: 0.857813
[[4160  801]
 [ 653 4386]]


In [44]:
evaluate(model_SKIPporter, Xp_test, yp_test)

313/313 [==============================] - 8s 24ms/step
Accuracy: 0.862700
Precision: 0.850612
Recall: 0.882516
F1 score: 0.866271
[[4180  781]
 [ 592 4447]]


In [45]:
evaluate(model_SKIPsnowball, Xs_test, ys_test)

313/313 [==============================] - 7s 23ms/step
Accuracy: 0.857500
Precision: 0.841201
Recall: 0.884104
F1 score: 0.862119
[[4120  841]
 [ 584 4455]]


In [46]:
evaluate(model_SKIPlancaster, Xla_test, yla_test)

313/313 [==============================] - 7s 22ms/step
Accuracy: 0.807400
Precision: 0.743089
Recall: 0.944235
F1 score: 0.831673
[[3316 1645]
 [ 281 4758]]


In [47]:
evaluate(model_SKIPlemma, Xle_test, yle_test)

313/313 [==============================] - 7s 22ms/step
Accuracy: 0.857400
Precision: 0.878483
Recall: 0.832110
F1 score: 0.854668
[[4381  580]
 [ 846 4193]]
